In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from tensorflow import keras

In [2]:
df = pd.read_csv("./wavefront_reconstruction_with_artificial.csv")
df.head()

,id,incoming_wavefront_x,incoming_wavefront_y,output_x,output_y,avg_rms_error
0,1,-0.97708,0.03072,0.00167,0.000011,0.045974
1,2,-0.97360,-0.06114,0.00370,0.000013,0.020102
2,3,-0.96199,-0.09952,0.00662,0.000006,0.032828
3,4,-0.95445,-0.03440,0.01025,0.000009,0.068953
4,5,-0.95445,0.10340,0.01564,0.000017,0.137871


In [3]:
df['avg_rms_error'].value_counts()

avg_rms_error
0.246824    6
0.283160    6
0.045974    3
0.325318    3
0.041868    3
           ..
0.684931    3
0.504976    3
0.510791    3
0.053546    3
0.000023    3
Name: count, Length: 1706, dtype: int64

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['avg_rms_error'] = le.fit_transform(df['avg_rms_error'])
df.head()

,id,incoming_wavefront_x,incoming_wavefront_y,output_x,output_y,avg_rms_error
0,1,-0.97708,0.03072,0.00167,0.000011,271
1,2,-0.97360,-0.06114,0.00370,0.000013,118
2,3,-0.96199,-0.09952,0.00662,0.000006,196
3,4,-0.95445,-0.03440,0.01025,0.000009,400
4,5,-0.95445,0.10340,0.01564,0.000017,691


In [5]:
affecting_parameter_name = le.classes_

In [6]:
X = df.drop(columns=['id', 'avg_rms_error'])
y = df['avg_rms_error']
X.head(3)

,incoming_wavefront_x,incoming_wavefront_y,output_x,output_y
0,-0.97708,0.03072,0.00167,0.000011
1,-0.97360,-0.06114,0.00370,0.000013
2,-0.96199,-0.09952,0.00662,0.000006


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=69)

In [8]:
x_train.shape

(3586, 4)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
print(x_train[:1])
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
print(x_train[:1])
print(x_test[:1])

     incoming_wavefront_x  incoming_wavefront_y  output_x  output_y
225              -0.74442              -0.38674   0.05799  0.000156
[[-1.20874035 -0.66833306  0.00574932 -0.32286095]]
[[-0.14044499  1.02856701 -1.01247772 -1.09731233]]


In [10]:
y_train = keras.utils.to_categorical(y_train, num_classes=1706)
print(y_train[:5])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(x_train.shape[-1], )))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=256, activation='relu'))  # Additional hidden layer
model.add(Dense(units=128, activation='relu'))   # Another additional hidden layer
model.add(Dense(units=64, activation='relu'))   # You can add more layers if needed
model.add(Dropout(0.5))
model.add(Dense(units=1706, activation='softmax'))

In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 256)               16640     
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1706)              1

In [14]:
model.fit(x_train, y_train, epochs=1000, verbose=2)

Epoch 1/1000


113/113 - 1s - loss: 7.4481 - accuracy: 2.7886e-04 - 842ms/epoch - 7ms/step
Epoch 2/1000
113/113 - 0s - loss: 6.9916 - accuracy: 0.0028 - 197ms/epoch - 2ms/step
Epoch 3/1000
113/113 - 0s - loss: 6.4075 - accuracy: 0.0078 - 198ms/epoch - 2ms/step
Epoch 4/1000
113/113 - 0s - loss: 5.9789 - accuracy: 0.0145 - 199ms/epoch - 2ms/step
Epoch 5/1000
113/113 - 0s - loss: 5.6663 - accuracy: 0.0142 - 193ms/epoch - 2ms/step
Epoch 6/1000
113/113 - 0s - loss: 5.4363 - accuracy: 0.0220 - 193ms/epoch - 2ms/step
Epoch 7/1000
113/113 - 0s - loss: 5.2246 - accuracy: 0.0279 - 193ms/epoch - 2ms/step
Epoch 8/1000
113/113 - 0s - loss: 5.0339 - accuracy: 0.0290 - 211ms/epoch - 2ms/step
Epoch 9/1000
113/113 - 0s - loss: 4.8934 - accuracy: 0.0335 - 193ms/epoch - 2ms/step
Epoch 10/1000
113/113 - 0s - loss: 4.7563 - accuracy: 0.0321 - 194ms/epoch - 2ms/step
Epoch 11/1000
113/113 - 0s - loss: 4.6339 - accuracy: 0.0413 - 198ms/epoch - 2ms/step
Epoch 12/1000
113/113 - 0s - loss: 4.5805 - accuracy: 0.0390 - 197ms/epo

In [ ]:
prediction = model.predict(x_test)

In [ ]:
model.save('laser_beam_model.h5', model)

c:\Users\ashen\anaconda3\envs\detection\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from tensorflow import keras
from keras.models import load_model
df = pd.read_csv("./wavefront_reconstruction_with_artificial.csv")
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
le = LabelEncoder()
df['avg_rms_error'] = le.fit_transform(df['avg_rms_error'])
X = df.drop(columns=['id', 'avg_rms_error'])
y = df['avg_rms_error']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=69)
x_train = sc.fit_transform(x_train)
model = load_model('laser_beam_model.h5')
affecting_parameter_name = le.classes_


# Initialize an empty list to store the captured values
captured_values = []

# Define a while loop that continues indefinitely
while True:
    # Prompt the user for input
    message = input("Enter a value (or type 'done' to finish): ")

    # Check if the user wants to exit the loop
    if message.lower() == 'done':
        break  # Exit the loop if the user types 'done'

    # Convert the input value to the appropriate data type and append it to the list
    try:
        value = float(message)  # Convert input to float
        captured_values.append(value)
    except ValueError:
        print("Invalid input. Please enter a valid number.")

# Ensure that exactly four values were captured
if len(captured_values) == 4:  # Check if exactly five values were captured
    # Assign the captured values to data1, data2, data3, and data4
    data1, data2, data3, data4 = captured_values
    X_new = np.array([[data1 , data2, data3, data4 ]])
    x_test = sc.transform(X_new)
    prediction = model.predict(x_test)
    prediction = np.argmax(prediction, axis=-1)
    error = (affecting_parameter_name[prediction[:1][0]]/0.2853)*14
    print("Most Affecting Error:",error,"%")
    # x1_test = sc.transform([[data1, data2, data3, data4, data5]])
    
    # Use the model to make predictions
    # pred = model.predict([[data1, data2, data3, data4, data5]])
    # pred_name = np.argmax(pred, axis=-1)
    
    # Print the predictions
    # print([[data1, data2, data3, data4, data5]].shape())
    # print("Predictions:", pred_name)
else:
    print("Please enter exactly five values.")


C:\Users\ashen\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 180ms/step
Most Affecting Error: 23.56694514546092 %
